# Detection 03 — Multi-Signal Email Risk

## Phase
Phase 4 — Detection Engineering

## Objective
Detect high-confidence phishing risk by correlating:
- Rare or first-seen sender domains
- Off-hours or unusual timing
- Privileged recipient roles

This detection reduces false positives by requiring multiple weak signals to align.


In [1]:
import pandas as pd
from pathlib import Path


In [2]:
PROJECT_ROOT = Path(r"D:\soc-dashboard-suite-main\soc-dashboard-suite-main")

INPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "email_with_temporal_context.csv"
ALERT_OUTPUT_PATH = PROJECT_ROOT / "data" / "enriched" / "alerts_multi_signal_email_risk.csv"

email_df = pd.read_csv(INPUT_PATH, parse_dates=["event_time"])


In [3]:
email_df["external_sender"] = email_df["sender_domain"] != email_df["recipient_domain"]


In [4]:
domain_risk = (
    (email_df["domain_rarity"] == "rare") |
    (email_df["is_first_seen_day"] == True)
)

time_risk = email_df["time_behavior"].isin(["off_hours", "unusual_hour"])
identity_risk = email_df["user_role"].isin(["admin", "executive"])
external_risk = email_df["external_sender"] == True

alerts_df = email_df[domain_risk & time_risk & identity_risk & external_risk].copy()

len(alerts_df)


3929

In [5]:
alerts_df["detection_id"] = "DET_03_MULTI_SIGNAL_EMAIL_RISK"
alerts_df["severity"] = "critical"

alerts_df["alert_reason"] = (
    "Rare/first-seen external domain + unusual timing + privileged recipient"
)


In [6]:
alert_fields = [
    "event_time",
    "sender_email",
    "sender_domain",
    "recipient_email",
    "user_role",
    "domain_rarity",
    "is_first_seen_day",
    "time_behavior",
    "severity",
    "alert_reason",
    "detection_id"
]

alerts_df = alerts_df[alert_fields]
alerts_df.head()


,event_time,sender_email,sender_domain,recipient_email,user_role,domain_rarity,is_first_seen_day,time_behavior,severity,alert_reason,detection_id
612,NaT,aod@newsdata.com,newsdata.com,western.price.survey.contacts@ren-6.cais.net,admin,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK
1434,NaT,aod@newsdata.com,newsdata.com,western.price.survey.contacts@ren-6.cais.net,admin,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK
2014,NaT,aod@newsdata.com,newsdata.com,western.price.survey.contacts@ren-6.cais.net,admin,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK
3912,NaT,info@amazon.com,amazon.com,jarnold@enron.com,executive,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK
3948,NaT,msagel@home.com,home.com,jarnold@enron.com,executive,rare,False,off_hours,critical,Rare/first-seen external domain + unusual timi...,DET_03_MULTI_SIGNAL_EMAIL_RISK


In [7]:
alerts_df.to_csv(ALERT_OUTPUT_PATH, index=False)
print("Saved alerts to:", ALERT_OUTPUT_PATH)


Saved alerts to: D:\soc-dashboard-suite-main\soc-dashboard-suite-main\data\enriched\alerts_multi_signal_email_risk.csv


In [8]:
len(alerts_df) / len(email_df)


0.007928500223991735